In [1]:
import pandas as pd
import numpy as np
from gtfparse import read_gtf
import functools as ft
import warnings
warnings.filterwarnings('ignore')

## Genes

In [2]:
# Read GTF with gene annotation
genes = read_gtf('/DATA/users/magnitov/tacl/genome/gencode.v44.annotation.gtf')
genes = genes[genes['feature'] == 'gene']
genes = genes[['seqname', 'start', 'end', 'strand', 'gene_id', 'gene_type', 'gene_name']]

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'tag', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'hgnc_id', 'havana_gene', 'ont', 'protein_id', 'ccdsid', 'artif_dupl']


In [3]:
def get_counts_table(genes, sample, replicate):
    counts_fwd = pd.read_csv('/DATA/users/magnitov/tacl/bruseq/counts/counts_forward.TACL_BrUseq.txt', 
                         names = ['gene_id', 
                                  'T-MAU2_NT_rep1', 'T-MAU2_NT_rep2', 'T-MAU2_NT_rep3', 
                                  'T-MAU2_Dox2h_rep1', 'T-MAU2_Dox2h_rep2', 'T-MAU2_Dox2h_rep3',
                                  'T-mCherry_NT_rep1', 'T-mCherry_NT_rep2', 'T-mCherry_NT_rep3',
                                  'T-mCherry_Dox2h_rep1', 'T-mCherry_Dox2h_rep2', 'T-mCherry_Dox2h_rep3'],
                         sep = '\t', header = None)
    counts_fwd = counts_fwd[counts_fwd['gene_id'].isin([x for x in counts_fwd['gene_id'] if x[0]!='_'])]
    counts_fwd = counts_fwd[['gene_id', sample + '_' + replicate]]
    
    counts_rev = pd.read_csv('/DATA/users/magnitov/tacl/bruseq/counts/counts_reverse.TACL_BrUseq.txt', 
                         names = ['gene_id', 
                                  'T-MAU2_NT_rep1', 'T-MAU2_NT_rep2', 'T-MAU2_NT_rep3', 
                                  'T-MAU2_Dox2h_rep1', 'T-MAU2_Dox2h_rep2', 'T-MAU2_Dox2h_rep3',
                                  'T-mCherry_NT_rep1', 'T-mCherry_NT_rep2', 'T-mCherry_NT_rep3',
                                  'T-mCherry_Dox2h_rep1', 'T-mCherry_Dox2h_rep2', 'T-mCherry_Dox2h_rep3'],
                         sep = '\t', header = None)
    counts_rev = counts_rev[counts_rev['gene_id'].isin([x for x in counts_rev['gene_id'] if x[0]!='_'])]
    counts_rev = counts_rev[['gene_id', sample + '_' + replicate]]
    
    genes_merged = genes.merge(counts_fwd, on = 'gene_id', how = 'left')
    genes_merged = genes_merged.merge(counts_rev, on = 'gene_id', how = 'left')
    
    counts_new = []
    for gene in genes_merged.values:
        if gene[3] == '+':
            counts_new.append(gene[7])
        else:
            counts_new.append(gene[8])
    genes_merged[f'{sample}_{replicate}'] = counts_new
    
    genes_merged = genes_merged.fillna(0)
    
    genes_merged = genes_merged[['seqname', 'start', 'end', 'strand', 'gene_id', 'gene_type', 'gene_name', f'{sample}_{replicate}']]
    
    return(genes_merged)

In [7]:
counts_T_MAU2_rep1 = get_counts_table(genes, 'T-MAU2_NT', 'rep1')
counts_T_MAU2_rep2 = get_counts_table(genes, 'T-MAU2_NT', 'rep2')
counts_T_MAU2_rep3 = get_counts_table(genes, 'T-MAU2_NT', 'rep3')
counts_T_mCherry_rep1 = get_counts_table(genes, 'T-mCherry_NT', 'rep1')
counts_T_mCherry_rep2 = get_counts_table(genes, 'T-mCherry_NT', 'rep2')
counts_T_mCherry_rep3 = get_counts_table(genes, 'T-mCherry_NT', 'rep3')
counts_T_MAU2_Dox_rep1 = get_counts_table(genes, 'T-MAU2_Dox2h', 'rep1')
counts_T_MAU2_Dox_rep2 = get_counts_table(genes, 'T-MAU2_Dox2h', 'rep2')
counts_T_MAU2_Dox_rep3 = get_counts_table(genes, 'T-MAU2_Dox2h', 'rep3')
counts_T_mCherry_Dox_rep1 = get_counts_table(genes, 'T-mCherry_Dox2h', 'rep1')
counts_T_mCherry_Dox_rep2 = get_counts_table(genes, 'T-mCherry_Dox2h', 'rep2')
counts_T_mCherry_Dox_rep3 = get_counts_table(genes, 'T-mCherry_Dox2h', 'rep3')

count_tables = [counts_T_MAU2_rep1, counts_T_MAU2_rep2, counts_T_MAU2_rep3, 
                counts_T_MAU2_Dox_rep1, counts_T_MAU2_Dox_rep2, counts_T_MAU2_Dox_rep3,
                counts_T_mCherry_rep1, counts_T_mCherry_rep2, counts_T_mCherry_rep3,
                counts_T_mCherry_Dox_rep1, counts_T_mCherry_Dox_rep2, counts_T_mCherry_Dox_rep3]
counts_tacl = ft.reduce(lambda left, right: pd.merge(left, right, on = ['seqname', 'start', 'end', 'strand', 'gene_id', 'gene_type', 'gene_name']), count_tables)
for column in counts_tacl.columns[7:]:
    counts_tacl[column] = [int(x) for x in counts_tacl[column]]
counts_tacl.head()

,seqname,start,end,strand,gene_id,gene_type,gene_name,T-MAU2_NT_rep1,T-MAU2_NT_rep2,T-MAU2_NT_rep3,T-MAU2_Dox2h_rep1,T-MAU2_Dox2h_rep2,T-MAU2_Dox2h_rep3,T-mCherry_NT_rep1,T-mCherry_NT_rep2,T-mCherry_NT_rep3,T-mCherry_Dox2h_rep1,T-mCherry_Dox2h_rep2,T-mCherry_Dox2h_rep3
0,chr1,11869,14409,+,ENSG00000290825.1,lncRNA,DDX11L2,0,0,1,0,0,0,0,0,0,0,0,0
1,chr1,12010,13670,+,ENSG00000223972.6,transcribed_unprocessed_pseudogene,DDX11L1,0,0,0,0,0,0,0,0,0,0,0,0
2,chr1,14404,29570,-,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,47,22,31,22,28,31,10,30,37,26,12,10
3,chr1,17369,17436,-,ENSG00000278267.1,miRNA,MIR6859-1,7,1,2,3,4,4,0,4,0,3,1,1
4,chr1,29554,31109,+,ENSG00000243485.5,lncRNA,MIR1302-2HG,0,0,1,0,0,0,0,0,0,0,0,0


In [8]:
counts_tacl[counts_tacl['gene_id'] == 'ENSG00000000460.17']

,seqname,start,end,strand,gene_id,gene_type,gene_name,T-MAU2_NT_rep1,T-MAU2_NT_rep2,T-MAU2_NT_rep3,T-MAU2_Dox2h_rep1,T-MAU2_Dox2h_rep2,T-MAU2_Dox2h_rep3,T-mCherry_NT_rep1,T-mCherry_NT_rep2,T-mCherry_NT_rep3,T-mCherry_Dox2h_rep1,T-mCherry_Dox2h_rep2,T-mCherry_Dox2h_rep3
4016,chr1,169662007,169854080,+,ENSG00000000460.17,protein_coding,C1orf112,2061,1331,2117,1098,1504,2040,705,2013,2158,291,434,1094


In [9]:
counts_tacl.drop(['seqname', 'start', 'end', 'strand', 'gene_type', 'gene_name'], axis = 1).to_csv('/DATA/users/magnitov/tacl/bruseq/counts/counts_deseq2.TACL.txt', sep = '\t', header = 0, index = 0)
counts_tacl.to_csv('/DATA/users/magnitov/tacl/bruseq/counts/counts_analysis.TACL.txt', sep = '\t', header = 1, index = 0)